In [68]:
import numpy as np
import cv2

# Définition des paramètres intrasèques de la caméra
  # focale
fx = 60
fy = 40
  # centre
cx = 0
cy = 0

A = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]) # matrice intrasèque de la caméra (3*3)

# Génération de 3 angles aléatoire pour définir la rotation
rax,pitch,yaw = np.random.uniform(0,2*np.pi,size=3)

# Génération de la matrice de rotation

Rx = np.array([[1,0,0],[0, np.cos(rax), -np.sin(rax)],[0, np.sin(rax), np.cos(rax)]])
Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],[0,1,0],[-np.sin(pitch),0, np.cos(pitch)]])
Rz = np.array([[np.cos(yaw), -np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0,0,1]])

R = Rz @ Ry @ Rx  # matrice de rotation (3*3)

# Génération de la translation de la caméra
T = np.random.uniform(-500,500,size=3)     # T = [tx,ty,tz]  (1*3)
T = T.reshape((3,1))                       # (3*1)

print(T)

[[  81.98046562]
 [ 162.418685  ]
 [-303.97712757]]


In [69]:
# Projection d'un point 3D vers 2D
def projection3D2D(point3D,T,R,A) :
  # point 3D = [ Xw, Yw, Zw ]'   (1*3)
  # T : matrice de translation de la caméra : (3*1)
  # R : matrice de rotation : (3*3)
  # A : matrice intrasèque de la caméra : (3*3)

  PI = np.concatenate((np.eye(3),np.zeros((3,1))),axis=1)  # (3*4)

  Rt = np.concatenate((R,T),axis=1)               # (3*4)
  Rt = np.concatenate((Rt,np.array([[0,0,0,1]])),axis=0)   # (4*4)

  point3D_bis = np.concatenate((np.reshape(point3D,(3,1)),np.array([[1]])),axis=0)   #(4*1)

  point2D = A @ PI @ Rt @ point3D_bis   # point 2D = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # point 2D = [u, v, 1] (3*1)
  return point2D[:2]

pt3D = [[3],[1],[4]]
print("point 3D :\n ", pt3D)
pt2D = projection3D2D(pt3D,T,R,A)
print("point 2D :\n", pt2D)

point 3D :
  [[3], [1], [4]]
point 2D :
 [[-16.92188449]
 [-21.75827225]]


In [70]:
# Création des couples de point 2D,3D
def point3Daleatoire() :
  return np.array([[np.random.uniform(-500,500),np.random.uniform(-500,500),np.random.uniform(-500,500)]])

# points 3D
P1 = point3Daleatoire()     # (1*3) -> pour P3P
P2 = point3Daleatoire()
P3 = point3Daleatoire()
P4 = point3Daleatoire()

print(np.shape(P1))
print("P1\n",P1)
points3D = np.concatenate((P1,P2,P3),axis=0);     # (3*3)
print("pt3D\n",points3D)

# point 2D
p1 = projection3D2D(P1,T,R,A)   # (2*1)
p1 = np.reshape(p1,(1,2))       # (1,2)
p2 = projection3D2D(P2,T,R,A)
p2 = np.reshape(p2,(1,2))
p3 = projection3D2D(P3,T,R,A)
p3 = np.reshape(p3,(1,2))
p4 = projection3D2D(P4,T,R,A)   # (2,1)


points2D = np.concatenate((p1,p2,p3),axis=0);    # (3*2)
print("p1\n",p1)
print("pt2D\n",points2D)
print(np.shape(points2D))
# Application de solveP3P pour retrouver la position de la caméra
retval, rvec, tvecs =  cv2.solveP3P(points3D,points2D,A,None, flags = cv2.SOLVEPNP_P3P)


# Mesure de l'erreur d'estimation
def erreur_estimation(R,T,R_P3P,T_P3P):
  erreur = 0
  for i in range(np.shape(R,1)) :
    for j in range(np.shape(R,2)):
      erreur += (R[i,j] - R_P3P[i,j])**2
  for i in range(np.shape(T,1)):
    erreur += (T[i] - T_P3P[i])**2
  return np.sqrt(erreur)


(1, 3)
P1
 [[380.78965377 -20.17544536 421.3249139 ]]
pt3D
 [[ 380.78965377  -20.17544536  421.3249139 ]
 [-360.91906534 -270.25991505 -349.79562558]
 [-353.7675273  -297.13961291  342.47858672]]
p1
 [[-60.72714633 -38.59186028]]
pt2D
 [[-60.72714633 -38.59186028]
 [ 41.71453055  35.2056122 ]
 [-87.1936937   36.13481253]]
(3, 2)
